In [1]:
import tic_tac_toe as game
import random
import numpy as np
from collections import deque

from keras.models import model_from_json
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.optimizers import SGD , Adam

Using TensorFlow backend.


In [2]:
N_ACTIONS = 36
GAMMA = 0.99 # decay rate of past observations
OBSERVATION = 3200. # timesteps to observe before training
FINAL_EPSILON = 0.0001 # final value of epsilon
INITIAL_EPSILON = 0.1 # starting value of epsilon
REPLAY_MEMORY = 5000 # number of previous transitions to remember
BATCH = 32 # size of minibatch
LEARNING_RATE = 1e-4

In [3]:
def build_model():
    model = Sequential()
    model.add(Dense(36, input_dim=36, activation='relu'))
    model.add(Dense(36, activation='relu'))
    model.add(Dense(36, activation='relu'))
    model.add(Dense(36, activation='relu'))
    model.add(Dense(36, activation='relu'))
    
    adam = Adam(lr=LEARNING_RATE)
    model.compile(loss='mse',optimizer=adam)
    return model

In [6]:
def train_network(model, args):
    game_state = game.ofttt()
    RM = deque()
    (x_t, r_0, terminal) = (game_state.initial, 0, False)
    s_t = np.array(x_t[1])
    #player2 = game.random_player(game_state, game_state.initial)
    if args == 'run':
        OBSERVE = 999999999    #Keep observe, never train
        epsilon = FINAL_EPSILON    #Use a small epsilon to choose mainly policy actions
        #Load model
        print ("Now we load weight")  
        model.load_weights("model.h5")
        adam = Adam(lr=LEARNING_RATE)
        model.compile(loss='mse',optimizer=adam)
        print ("Weight load successfully")
    else:
        #Assign an observation variable and max epsilon to train
        OBSERVE = OBSERVATION
        epsilon = INITIAL_EPSILON
        
    t = 0
    
    while(True and t < 10000):
        
        if terminal:
            print "------------------------------------------FINAL-----------------------------------------------"
            print "Reward:",r_t
            game_state.display(x_t)
            (x_t, r_0, terminal) = (game_state.initial, 0, False)
            #game_state = game_state.initial         
        
        #Initialize variables
        loss = 0
        Q_sa = 0
        action_index = 0
        r_t = 0
        a_t = np.zeros([N_ACTIONS])    #Output vector of actions a_[t] = 1 for action to take
        
        player = x_t[0]
        
        if player == 1:
            if random.random() <= epsilon:    #At the first move, choose randomly
                print("----------Random Action----------")
                action_index = random.randrange(N_ACTIONS)
                a_t[action_index] = 1
            else:
                q = model.predict(s_t.reshape(1,36))       #input the state at time t
                max_Q = np.argmax(q)         #Take the max q value predicted from network
                action_index = 1 + max_Q         #Assign action to the argmax Q
                a_t[max_Q] = 1               #Output vector a_t = 1 for max_Q

            #Decrease epsilon by a smalll factor
            if epsilon > FINAL_EPSILON and t > OBSERVE:               
                epsilon -= (INITIAL_EPSILON - FINAL_EPSILON) / 10000
            
            moves = game_state.legal_moves(x_t)
            if action_index not in moves:
                #print action_index, moves
                terminal = True
                r_t = -1
            else:
                x_t1, r_t, terminal = game_state.next_state(action_index, x_t) #run the selected action and observed next state and reward
            #print " 1:", r_t, action_index#, x_t[1]
            
            
        else:
            #Random agent
            moves = game_state.legal_moves(x_t)
            randomAction = moves[random.randrange(0, len(moves))]
            x_t1, r_t, terminal = game_state.next_state(randomAction, x_t)
            #print "-1:", r_t, randomAction#, x_t[1]            
            
        s_t1 = np.array(x_t1[1])
        RM.append((s_t, action_index, r_t, s_t1, terminal))    # store the transition in the Replay Memory
        if len(RM) > REPLAY_MEMORY:
            RM.popleft()

        #only train if done observing
        if t > OBSERVE:
            #sample a minibatch to train on
            minibatch = random.sample(RM, BATCH)
            inputs = np.zeros((BATCH, N_ACTIONS))  
            #print (inputs.shape)
            targets = np.zeros((inputs.shape[0], N_ACTIONS)) 

            #Now we do the experience replay
            for i in range(0, len(minibatch)):
                x0_t = np.array(minibatch[i][0])
                action_t = minibatch[i][1]   #This is action index
                reward_t = minibatch[i][2]
                state_t1 = minibatch[i][3]
                terminal = minibatch[i][4]
                # if terminated, only equals reward
                state_t = x0_t.reshape(1,N_ACTIONS)

                inputs[i:i + 1] = state_t    #I saved down s_t

                targets[i] = model.predict(state_t)  # Hitting each buttom probability
                Q_sa = model.predict(state_t1.reshape(1,N_ACTIONS))

                if terminal:
                    targets[i, action_t] = reward_t
                else:
                    targets[i, action_t] = reward_t + GAMMA * np.max(Q_sa)

            # targets2 = normalize(targets)
            loss += model.train_on_batch(inputs, targets)

            
        moves = game_state.legal_moves(x_t)
        #print "Moves: ", moves      
        player = x_t1[0]        
        s_t = s_t1
        x_t = x_t1
        t = t + 1
        
        # save progress every 10000 iterations
        if t % 1000 == 0:
            print("Now we save model")
            model.save_weights("model.h5", overwrite=True)

        # print info
        info = ""
        if t <= OBSERVE:
            info = "observe"
        else:
            info = "train"

        print("TIMESTEP", t, "/ STATE", info, "/ EPSILON", epsilon, "/ ACTION", action_index, "/ REWARD", r_t, "/ Q_MAX " , np.max(Q_sa), "/ Loss ", loss)
       
    print("Episode finished!")
    print("************************")

In [7]:
def playGame(args):
    model = build_model()
    train_network(model,args)

In [8]:
def main():
    #parser = argparse.ArgumentParser(description='Description of your program')
    #parser.add_argument('-m','--mode', help='Train / Run', required=True)
    #args = vars(parser.parse_args())
    playGame(args="train")

In [9]:
if __name__ == "__main__":
    main()

('TIMESTEP', 1, '/ STATE', 'observe', '/ EPSILON', 0.1, '/ ACTION', 1, '/ REWARD', 0, '/ Q_MAX ', 0, '/ Loss ', 0)
('TIMESTEP', 2, '/ STATE', 'observe', '/ EPSILON', 0.1, '/ ACTION', 0, '/ REWARD', 0, '/ Q_MAX ', 0, '/ Loss ', 0)
('TIMESTEP', 3, '/ STATE', 'observe', '/ EPSILON', 0.1, '/ ACTION', 27, '/ REWARD', 0, '/ Q_MAX ', 0, '/ Loss ', 0)
('TIMESTEP', 4, '/ STATE', 'observe', '/ EPSILON', 0.1, '/ ACTION', 0, '/ REWARD', 0, '/ Q_MAX ', 0, '/ Loss ', 0)
('TIMESTEP', 5, '/ STATE', 'observe', '/ EPSILON', 0.1, '/ ACTION', 30, '/ REWARD', 0, '/ Q_MAX ', 0, '/ Loss ', 0)
('TIMESTEP', 6, '/ STATE', 'observe', '/ EPSILON', 0.1, '/ ACTION', 0, '/ REWARD', 0, '/ Q_MAX ', 0, '/ Loss ', 0)
('TIMESTEP', 7, '/ STATE', 'observe', '/ EPSILON', 0.1, '/ ACTION', 30, '/ REWARD', -1, '/ Q_MAX ', 0, '/ Loss ', 0)
------------------------------------------FINAL-----------------------------------------------
Reward: -1
 0  1  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
-1  0  0  1  

 0  1  0  0 -1  1
 0  0  0  0  0  0
 0  0  0  0  0  1
-1  0 -1  0  0  0
('TIMESTEP', 164, '/ STATE', 'observe', '/ EPSILON', 0.1, '/ ACTION', 17, '/ REWARD', 0, '/ Q_MAX ', 0, '/ Loss ', 0)
('TIMESTEP', 165, '/ STATE', 'observe', '/ EPSILON', 0.1, '/ ACTION', 0, '/ REWARD', 0, '/ Q_MAX ', 0, '/ Loss ', 0)
('TIMESTEP', 166, '/ STATE', 'observe', '/ EPSILON', 0.1, '/ ACTION', 27, '/ REWARD', 0, '/ Q_MAX ', 0, '/ Loss ', 0)
('TIMESTEP', 167, '/ STATE', 'observe', '/ EPSILON', 0.1, '/ ACTION', 0, '/ REWARD', 0, '/ Q_MAX ', 0, '/ Loss ', 0)
('TIMESTEP', 168, '/ STATE', 'observe', '/ EPSILON', 0.1, '/ ACTION', 13, '/ REWARD', 0, '/ Q_MAX ', 0, '/ Loss ', 0)
('TIMESTEP', 169, '/ STATE', 'observe', '/ EPSILON', 0.1, '/ ACTION', 0, '/ REWARD', 0, '/ Q_MAX ', 0, '/ Loss ', 0)
('TIMESTEP', 170, '/ STATE', 'observe', '/ EPSILON', 0.1, '/ ACTION', 13, '/ REWARD', -1, '/ Q_MAX ', 0, '/ Loss ', 0)
------------------------------------------FINAL-----------------------------------------------
Reward: -

 0  0  0  0  0  0
 0  0  0  0  0  0
 1  0  0  0  0  0
('TIMESTEP', 307, '/ STATE', 'observe', '/ EPSILON', 0.1, '/ ACTION', 15, '/ REWARD', 0, '/ Q_MAX ', 0, '/ Loss ', 0)
('TIMESTEP', 308, '/ STATE', 'observe', '/ EPSILON', 0.1, '/ ACTION', 0, '/ REWARD', 0, '/ Q_MAX ', 0, '/ Loss ', 0)
('TIMESTEP', 309, '/ STATE', 'observe', '/ EPSILON', 0.1, '/ ACTION', 27, '/ REWARD', 0, '/ Q_MAX ', 0, '/ Loss ', 0)
('TIMESTEP', 310, '/ STATE', 'observe', '/ EPSILON', 0.1, '/ ACTION', 0, '/ REWARD', 0, '/ Q_MAX ', 0, '/ Loss ', 0)
('TIMESTEP', 311, '/ STATE', 'observe', '/ EPSILON', 0.1, '/ ACTION', 30, '/ REWARD', 0, '/ Q_MAX ', 0, '/ Loss ', 0)
('TIMESTEP', 312, '/ STATE', 'observe', '/ EPSILON', 0.1, '/ ACTION', 0, '/ REWARD', 0, '/ Q_MAX ', 0, '/ Loss ', 0)
('TIMESTEP', 313, '/ STATE', 'observe', '/ EPSILON', 0.1, '/ ACTION', 19, '/ REWARD', -1, '/ Q_MAX ', 0, '/ Loss ', 0)
------------------------------------------FINAL-----------------------------------------------
Reward: -1
-1  0  0  0  0  

 0  0  0  0  0  0
 0  1  0  0  0  1
 0  1  0  0 -1  0
 0 -1 -1  1  0  0
 0  0  0  0  0  0
----------Random Action----------
('TIMESTEP', 458, '/ STATE', 'observe', '/ EPSILON', 0.1, '/ ACTION', 9, '/ REWARD', 0, '/ Q_MAX ', 0, '/ Loss ', 0)
('TIMESTEP', 459, '/ STATE', 'observe', '/ EPSILON', 0.1, '/ ACTION', 0, '/ REWARD', 0, '/ Q_MAX ', 0, '/ Loss ', 0)
('TIMESTEP', 460, '/ STATE', 'observe', '/ EPSILON', 0.1, '/ ACTION', 29, '/ REWARD', 0, '/ Q_MAX ', 0, '/ Loss ', 0)
('TIMESTEP', 461, '/ STATE', 'observe', '/ EPSILON', 0.1, '/ ACTION', 0, '/ REWARD', 0, '/ Q_MAX ', 0, '/ Loss ', 0)
('TIMESTEP', 462, '/ STATE', 'observe', '/ EPSILON', 0.1, '/ ACTION', 21, '/ REWARD', 0, '/ Q_MAX ', 0, '/ Loss ', 0)
('TIMESTEP', 463, '/ STATE', 'observe', '/ EPSILON', 0.1, '/ ACTION', 0, '/ REWARD', 0, '/ Q_MAX ', 0, '/ Loss ', 0)
('TIMESTEP', 464, '/ STATE', 'observe', '/ EPSILON', 0.1, '/ ACTION', 29, '/ REWARD', -1, '/ Q_MAX ', 0, '/ Loss ', 0)
------------------------------------------FINAL------

 1  0  0  0  0  0
('TIMESTEP', 614, '/ STATE', 'observe', '/ EPSILON', 0.1, '/ ACTION', 29, '/ REWARD', 0, '/ Q_MAX ', 0, '/ Loss ', 0)
('TIMESTEP', 615, '/ STATE', 'observe', '/ EPSILON', 0.1, '/ ACTION', 0, '/ REWARD', 0, '/ Q_MAX ', 0, '/ Loss ', 0)
('TIMESTEP', 616, '/ STATE', 'observe', '/ EPSILON', 0.1, '/ ACTION', 30, '/ REWARD', 0, '/ Q_MAX ', 0, '/ Loss ', 0)
('TIMESTEP', 617, '/ STATE', 'observe', '/ EPSILON', 0.1, '/ ACTION', 0, '/ REWARD', 0, '/ Q_MAX ', 0, '/ Loss ', 0)
('TIMESTEP', 618, '/ STATE', 'observe', '/ EPSILON', 0.1, '/ ACTION', 27, '/ REWARD', -1, '/ Q_MAX ', 0, '/ Loss ', 0)
------------------------------------------FINAL-----------------------------------------------
Reward: -1
 0  0 -1  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0 -1  0  1
 1  0  0  0  0  0
('TIMESTEP', 619, '/ STATE', 'observe', '/ EPSILON', 0.1, '/ ACTION', 27, '/ REWARD', 0, '/ Q_MAX ', 0, '/ Loss ', 0)
('TIMESTEP', 620, '/ STATE', 'observe', '/ EPSILON', 0.1, '/ 

...

('TIMESTEP', 8773, '/ STATE', 'train', '/ EPSILON', 0.058141899999994195, '/ ACTION', 0, '/ REWARD', 0, '/ Q_MAX ', 1078.6416, '/ Loss ', 5081.60546875)
------------------------------------------FINAL-----------------------------------------------
Reward: 0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0 -1
 0  0  1  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
----------Random Action----------
('TIMESTEP', 8774, '/ STATE', 'train', '/ EPSILON', 0.058131909999994194, '/ ACTION', 11, '/ REWARD', 0, '/ Q_MAX ', 1073.2344, '/ Loss ', 4481.5732421875)
------------------------------------------FINAL-----------------------------------------------
Reward: 0
 0  0  0  0  0  0
 0  0  0  0  0  1
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
('TIMESTEP', 8775, '/ STATE', 'train', '/ EPSILON', 0.05812191999999419, '/ ACTION', 20, '/ REWARD', 0, '/ Q_MAX ', 777.13641, '/ Loss ', 3818.507080078125)
('TIMESTEP', 8776, '/ STATE', 'train', '/ EPSILON', 0.05812191999999419, '

('TIMESTEP', 8817, '/ STATE', 'train', '/ EPSILON', 0.05780223999999415, '/ ACTION', 20, '/ REWARD', -1, '/ Q_MAX ', 1154.3971, '/ Loss ', 5366.517578125)
------------------------------------------FINAL-----------------------------------------------
Reward: -1
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  1  0  0  0
 0  0  0  0  0  0
 0  0 -1  0  0  0
('TIMESTEP', 8818, '/ STATE', 'train', '/ EPSILON', 0.05779224999999415, '/ ACTION', 20, '/ REWARD', 0, '/ Q_MAX ', 768.43549, '/ Loss ', 4388.67529296875)
('TIMESTEP', 8819, '/ STATE', 'train', '/ EPSILON', 0.05779224999999415, '/ ACTION', 0, '/ REWARD', 0, '/ Q_MAX ', 1128.2113, '/ Loss ', 6557.61474609375)
('TIMESTEP', 8820, '/ STATE', 'train', '/ EPSILON', 0.057782259999994146, '/ ACTION', 20, '/ REWARD', -1, '/ Q_MAX ', 867.56274, '/ Loss ', 4916.68701171875)
('TIMESTEP', 8821, '/ STATE', 'train', '/ EPSILON', 0.057772269999994144, '/ ACTION', 20, '/ REWARD', -1, '/ Q_MAX ', 1006.8785, '/ Loss ', 4914.4990234375)
-----

('TIMESTEP', 8854, '/ STATE', 'train', '/ EPSILON', 0.05753250999999411, '/ ACTION', 20, '/ REWARD', -1, '/ Q_MAX ', 1082.7772, '/ Loss ', 6481.66357421875)
('TIMESTEP', 8855, '/ STATE', 'train', '/ EPSILON', 0.05752251999999411, '/ ACTION', 20, '/ REWARD', -1, '/ Q_MAX ', 1087.6664, '/ Loss ', 5489.2783203125)
------------------------------------------FINAL-----------------------------------------------
Reward: -1
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  1  0  0  0
 0  0  0 -1  0  0
 0  0  0  0  0  0
('TIMESTEP', 8856, '/ STATE', 'train', '/ EPSILON', 0.05751252999999411, '/ ACTION', 20, '/ REWARD', 0, '/ Q_MAX ', 918.74738, '/ Loss ', 4270.0224609375)
('TIMESTEP', 8857, '/ STATE', 'train', '/ EPSILON', 0.05751252999999411, '/ ACTION', 0, '/ REWARD', 0, '/ Q_MAX ', 1035.6465, '/ Loss ', 4522.6171875)
('TIMESTEP', 8858, '/ STATE', 'train', '/ EPSILON', 0.05750253999999411, '/ ACTION', 20, '/ REWARD', -1, '/ Q_MAX ', 1014.6243, '/ Loss ', 5596.958984375)
------------

('TIMESTEP', 8892, '/ STATE', 'train', '/ EPSILON', 0.05726277999999407, '/ ACTION', 20, '/ REWARD', -1, '/ Q_MAX ', 1090.2167, '/ Loss ', 6366.908203125)
------------------------------------------FINAL-----------------------------------------------
Reward: -1
 0  0 -1  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  1  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
('TIMESTEP', 8893, '/ STATE', 'train', '/ EPSILON', 0.05725278999999407, '/ ACTION', 20, '/ REWARD', 0, '/ Q_MAX ', 794.95599, '/ Loss ', 7612.91259765625)
('TIMESTEP', 8894, '/ STATE', 'train', '/ EPSILON', 0.05725278999999407, '/ ACTION', 0, '/ REWARD', 0, '/ Q_MAX ', 637.31885, '/ Loss ', 7188.056640625)
------------------------------------------FINAL-----------------------------------------------
Reward: 0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0 -1  0  0
 0  0  1  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
('TIMESTEP', 8895, '/ STATE', 'train', '/ EPSILON', 0.05724279999999407, '/ ACTION', 20, '/ REWARD', 0, '/ Q_

KeyboardInterrupt: 

In [ ]:
print(game_state.next_state(22,(1,[[1, -1, -1, 0, 0, 0], [0, 1, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]])))

In [ ]:
print(game_state.initial)